In [5]:
# import necessary libraries
import skimage as ski
import pandas as pd 
import os 
from skimage.transform import resize 
from skimage.io import imread 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import svm 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report


In [6]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [7]:
# list all the folders in the train directory
Categories = os.listdir('train/')

In [10]:
flat_data_arr=[] #input array 
target_arr=[] #output array 
datadir='train/'
#path which contains all the categories of images 
for i in Categories: 
	
	print(f'loading... category : {i}') 
	path=os.path.join(datadir,i) 
	for img in os.listdir(path): 
		img_array=imread(os.path.join(path,img)) 
		img_resized=resize(img_array,(64,64,3)) 
		flat_data_arr.append(img_resized.flatten()) 
		target_arr.append(Categories.index(i)) 
	print(f'loaded category:{i} successfully') 
flat_data=np.array(flat_data_arr) 
target=np.array(target_arr)


loading... category : A
loaded category:A successfully
loading... category : B
loaded category:B successfully
loading... category : C
loaded category:C successfully
loading... category : D
loaded category:D successfully
loading... category : E
loaded category:E successfully
loading... category : F
loaded category:F successfully
loading... category : G
loaded category:G successfully
loading... category : H
loaded category:H successfully
loading... category : I
loaded category:I successfully
loading... category : K
loaded category:K successfully
loading... category : L
loaded category:L successfully
loading... category : M
loaded category:M successfully
loading... category : N
loaded category:N successfully
loading... category : O
loaded category:O successfully
loading... category : P
loaded category:P successfully
loading... category : Q
loaded category:Q successfully
loading... category : R
loaded category:R successfully
loading... category : S
loaded category:S successfully
loading...

In [12]:
from sklearn.decomposition import PCA
num_features = 10  # Number of features to extract
pca = PCA(n_components=num_features)  # PCA for feature extraction
# Perform PCA to extract 10 features
pca.fit(flat_data)
flat_data_pca = pca.transform(flat_data)

In [21]:
#dataframe 
df=pd.DataFrame(flat_data_pca) 
df['Target']=target 
# store the df in compressed format like hdf5
df.to_hdf('feature_data.h5', 'df', mode='w')


In [15]:
# 1. Divide dataframe into input and output
x = df.drop(columns=['Target']) # Drop the column to be predicted
y = df['Target'] # Choose Output column to be predicted
feature_names = x.columns

In [16]:
# Splitting the data into training and testing sets 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20, 
											random_state=77, 
											stratify=y)


In [17]:
# Defining the parameters grid for GridSearchCV 
param_grid={'C':[0.1,1,10,100], 
			'gamma':[0.0001,0.001,0.1,1], 
			'kernel':['rbf','poly']} 

# Creating a support vector classifier 
svc=svm.SVC(probability=True) 

# Creating a model using GridSearchCV with the parameters grid 
model=GridSearchCV(svc,param_grid)


In [18]:
# Training the model using the training data 
model.fit(x_train,y_train)

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.0001, 0.001, 0.1, 1],
                         'kernel': ['rbf', 'poly']})

In [20]:
# Save Model Using Pickle
import pickle
filename = 'svm_model.sav'
pickle.dump(model, open(filename, 'wb'))


# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.9939698492462311


In [19]:
# Testing the model using the testing data 
y_pred = model.predict(x_test) 

# Calculating the accuracy of the model 
accuracy = accuracy_score(y_pred, y_test) 

# Print the accuracy of the model 
print(f"The model is {accuracy*100}% accurate")

The model is 99.39698492462311% accurate


In [24]:
x_test

,0,1,2,3,4,5,6,7,8,9
2454,-4.167401,-2.165257,0.245013,-2.343775,4.894823,-2.033633,0.934225,0.561614,2.044304,1.521335
2099,-4.626113,6.105807,-4.300303,-0.828647,0.400587,-2.800704,0.149434,1.261171,0.229728,-1.146454
2878,-7.631816,-4.546678,-1.177251,-0.092722,3.264285,1.994451,3.883889,3.612116,3.336730,4.953765
3346,-2.527864,-2.893822,-2.060138,-4.409529,-1.574370,-4.739651,-2.883905,1.512568,-2.436245,0.994644
768,24.696443,2.213841,-3.985872,0.931312,-1.501287,0.072388,-0.720068,1.069958,-0.944023,0.268730
...,...,...,...,...,...,...,...,...,...,...
4266,8.305686,1.644209,-0.550333,3.566200,-8.206873,2.155667,-1.320852,0.303917,-3.849782,0.579415
4713,-2.281352,-4.891825,3.585142,-1.808665,-4.675589,0.959122,-0.415246,0.374758,-1.031158,0.430178
4216,9.220889,-3.657000,-2.290774,7.050332,-2.818508,4.158800,-1.561179,-2.224167,0.507906,-1.900934
612,-6.340750,-4.596051,2.599262,0.753465,3.046302,3.058570,1.300698,6.805993,1.012997,-1.592137
